# Sprint 公開されている実装を動かす

In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

Q1

In [ ]:


img = mpimg.imread(‘Sprint17_output/sample.jpg’)
imgplot = plt.imshow(img)

Q2

Q3

Q4

Q5

Q6

Q7

Q8

Q9

Q10

Q11